# Some notes on regression statistics

In [40]:
import numpy as np
import pandas as pd
from scipy.stats import t
import statsmodels.api as sm
import matplotlib.pyplot as plt
from IPython.display import display

# generate random regression data
N=1000
M=5
X = np.random.ranf((N,M))
B = np.random.ranf(M)
eps = np.random.normal(size=N) * 10
Y = X @ B + eps

In [43]:
# Estimate betas
reg = pd.DataFrame()
B_hat = np.linalg.inv(X.T @ X) @ X.T @ Y
reg['Beta hat'] = B_hat

# get residuals
eps_hat = Y - X @ B_hat

# Get R2
TSS = Y.T @ Y # total SS
RSS = eps_hat.T @ eps_hat.T # residual SS
R2 = 1-RSS/TSS
print(f'R2: [{R2:.3f}]')

# get log likelihood
E_HAT = Y - X @ B_hat
VAR_E = np.sum(E_HAT.T @ E_HAT)/(N-M) - (np.sum(E_HAT)/(N-M))**2
SD_E = VAR_E ** 0.5

# \frac{1}{\sigma (2 \pi) ^ 0.5} * \e^{-0.5 \frac{x-mu}{\sigma}^2}
loglik = np.sum(-0.5 * (E_HAT/SD_E)**2) + N * np.log(1/(SD_E * ((2*np.pi)**0.5)))
BIC = M * np.log(N) - 2 * loglik
print(f'log-likelihood [{loglik:.2f}]')
print(f'Bayesian information criteria: [{BIC:.2f}]')

# Get SE per beta
# 1: VAR(AY) = A VAR(Y) A^T
# 2: B_hat = (X^T X)^{-1} X^T Y
# substituting U = (X^T X)^{-1} X^T => VAR(B_hat) = VAR(U Y) = U VAR(Y) U^T
U = np.linalg.inv(X.T @ X) @ X.T
SE_beta = (U @ (np.identity(N) * VAR_E) @ U.T).diagonal() ** 0.5
reg['Beta SEs'] = SE_beta

# Get t-stat per beta
t_stats = B_hat / SE_beta
reg['t-stats'] = t_stats

# Get p-stat per beta
p_null = 1 - 2 * np.abs(t.cdf(t_stats, N-M) - 0.5)  # 1 - prob of being further from center
reg['p-vals'] = p_null

display(reg)

R2: [0.050]
log-likelihood [-3702.88]
Bayesian information criteria: [7440.31]


,Beta hat,Beta SEs,t-stats,p-vals
0,2.618201,1.014816,2.579977,0.010023
1,2.082878,0.965994,2.156203,0.031307
2,-0.955345,0.925653,-1.032076,0.302287
3,1.018135,0.964835,1.055242,0.291571
4,-0.745855,0.977628,-0.762923,0.445690


In [44]:
print(sm.OLS(Y, X).fit().summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.050
Model:                            OLS   Adj. R-squared (uncentered):              0.045
Method:                 Least Squares   F-statistic:                              10.44
Date:                Sat, 31 Jul 2021   Prob (F-statistic):                    8.75e-10
Time:                        20:52:12   Log-Likelihood:                         -3702.9
No. Observations:                1000   AIC:                                      7416.
Df Residuals:                     995   BIC:                                      7440.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------